In [35]:
import pandas as pd
import pickle
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
from sklearn.cluster import KMeans
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [36]:
#nn_model = pickle.load(open('model_Kaggle', 'rb'))


In [53]:
parms = {'max_depth':16, #maximum depth of a tree
         'objective':'reg:linear',
         'eta'      :0.1,
         'subsample':0.8,#SGD will use this percentage of data
         'lambda '  :1, #L2 regularization term,>1 more conservative 
         'colsample_bytree ':0.9,
         'colsample_bylevel':1,
         'min_child_weight': 10} 

xgb_model = xgb.Booster(parms)
xgb_model.load_model('model_Kaggle')

In [47]:
full_tripdata= pd.read_csv("data/test.csv")

In [48]:
full_tripdata.columns

Index([u'id', u'vendor_id', u'pickup_datetime', u'passenger_count',
       u'pickup_longitude', u'pickup_latitude', u'dropoff_longitude',
       u'dropoff_latitude', u'store_and_fwd_flag'],
      dtype='object')

In [49]:
#processing test data 

full_tripdata["pickup_datetime"] = pd.to_datetime(full_tripdata["pickup_datetime"])
full_tripdata = full_tripdata.drop(columns=['vendor_id','store_and_fwd_flag','passenger_count'])
full_tripdata["weekday"] = full_tripdata["pickup_datetime"].dt.dayofweek
full_tripdata["hourofday"] = full_tripdata["pickup_datetime"].dt.hour
#prepare location, use pre-trained k-mean model
model = pickle.load(open('location_cluster_kmean', 'rb'))
print full_tripdata.columns
pickupLocations = full_tripdata.iloc[:,2:4]
print pickupLocations.describe()
full_tripdata["pickupLoc"] = model.predict(pickupLocations.as_matrix())

dropoffLocations = full_tripdata.iloc[:,4:6]
full_tripdata["dropoffLoc"] = model.predict(dropoffLocations.as_matrix())
print dropoffLocations.describe()

training = full_tripdata.drop(columns=['pickup_datetime',
                                       'dropoff_longitude',
                                       'dropoff_latitude',
                                       'pickup_longitude',
                                       'pickup_latitude',
                                       'id'])

training['weekday'] = training['weekday'].astype('category')
training['hourofday'] = training['hourofday'].astype('category')
training['pickupLoc'] = training['pickupLoc'].astype('category')
training['dropoffLoc'] = training['dropoffLoc'].astype('category')

X = pd.get_dummies(training)

Index([u'id', u'pickup_datetime', u'pickup_longitude', u'pickup_latitude',
       u'dropoff_longitude', u'dropoff_latitude', u'weekday', u'hourofday'],
      dtype='object')
       pickup_longitude  pickup_latitude
count     625134.000000    625134.000000
mean         -73.973614        40.750927
std            0.073389         0.029848
min         -121.933128        37.389587
25%          -73.991852        40.737392
50%          -73.981743        40.754093
75%          -73.967400        40.768394
max          -69.248917        42.814938
       dropoff_longitude  dropoff_latitude
count      625134.000000     625134.000000
mean          -73.973458         40.751816
std             0.072565          0.035824
min          -121.933327         36.601322
25%           -73.991318         40.736000
50%           -73.979774         40.754543
75%           -73.963013         40.769852
max           -67.496796         48.857597


In [50]:
X.columns

Index([u'weekday_0', u'weekday_1', u'weekday_2', u'weekday_3', u'weekday_4',
       u'weekday_5', u'weekday_6', u'hourofday_0', u'hourofday_1',
       u'hourofday_2',
       ...
       u'dropoffLoc_40', u'dropoffLoc_41', u'dropoffLoc_42', u'dropoffLoc_43',
       u'dropoffLoc_44', u'dropoffLoc_45', u'dropoffLoc_46', u'dropoffLoc_47',
       u'dropoffLoc_48', u'dropoffLoc_49'],
      dtype='object', length=131)

In [58]:
data_test = xgb.DMatrix(X)
result = xgb_model.predict(data_test)
#nn_model.feature_types

In [59]:
ytest = np.exp(result)+1

In [60]:
submission = pd.DataFrame({'id': test.id, 'trip_duration': ytest})

In [61]:
submission.to_csv("submission.csv",index=False)